<h1> <center> Create dataset 4 ML classification </center> </h1>

In [166]:
import pandas as pd
import numpy as np 
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline 

in this notebook you will find : 

* **Step 1** : Define Features types > Categorical / Numerical / Text
* **Step 2** : Build the target > based on two columns `IMMEDIATE_EXTUBATION` and `SECONDARY_EXTUBATION`
* **Step 3** : Select a subset of patient > drop useless patients
* **Step 4** : Export the new data ready for feature engineering > folder : `data/data_merged.csv`

In [167]:
dim_patient_intraoperatoire = pd.read_csv("../../../data/dim_patient_intraoperatoire.csv")
dim_donneur = pd.read_csv("../../../data/dim_donneur.csv")
dim_patient_preoperatoire = pd.read_csv("../../../data/dim_patient_preoperatoire.csv")
dim_patient_postoperatoire = pd.read_csv("../../../data/dim_patient_postoperatoire.csv")

data = pd.merge(dim_patient_preoperatoire, dim_donneur, how = 'left', on = "numero")
data = pd.merge(data, dim_patient_postoperatoire[["numero", " immediate_extubation", "secondary_intubation"]], how = 'left', on = 'numero').rename(columns = {' immediate_extubation':  'immediate_extubation'})

# Define features type 

In [168]:
id_col = "numero"

cat_col = [
    "date_transplantation", 
    "heure_arrivee_bloc",
    "pathologie", 
    "age", 
    "sex", 
    "other_organ_transplantation", 
    "transplanted_twice_during_study_period", 
    "super_urgence", 
    "retransplant", 
    "preoperative_ICU", 
    "preoperative_vasopressor",
    "preoperative_mechanical_ventilation",
    "PFO", 
    "diabetes", 
    "preoperative_pulmonary_hypertension"
    "Insuffisance_renale", 
    "CMV_receveur", 
    "plasmapherese", 
    "preoperative_ECMO", 
    "ATCD_chirugicaux", 
    "thoracic_surgery_history", 
    "CMV_donneur", 
    "EBV_donneur", 
    "Sex_donor", 
    "BMI_donor"
]

num_col = [
    "Poids", 
    "Taille", 
    "time_on_waiting_liste", 
    "LAS", 
    "body_mass_index",
    "PAPS", 
    "Age_donor", 
    "Poids_donor", 
    "Taille_donor", 
    "Donneur_CPT",
    "Tabagisme_donor",
    "Aspirations_donor", 
    "RX_donor", 
    "PF_donor",
    "oto_score"   
]

text = [
    "ATCD_medicaux", 
    "ATCD_chirugicaux"
]
target = [
    'immediate_extubation', 
    'secondary_intubation'
    
]
col_drop = ["Unnamed: 0_x", "Unnamed: 0_y"]

## Build the target

In [1]:
data.drop(col_drop, axis = 1, inplace = True)
data.columns = [i.lower() for i in data.columns]

data["target"]  = np.nan
data["target"][(data["immediate_extubation"] == 1) & (data["secondary_intubation"] == 1)] = "unsuccessful IE"
data["target"][(data["immediate_extubation"] == 1) & (data["secondary_intubation"] == 0)] = "successful IE"

data[data.target.notnull()][['target', 'immediate_extubation', "secondary_intubation"]].head()

NameError: name 'data' is not defined

In [170]:
print("effectif unsuccessful IE {}".format(((data["immediate_extubation"] == 1) & (data["secondary_intubation"] == 1)).sum()))
print("effectif successful IE {}".format(((data["immediate_extubation"] == 1) & (data["secondary_intubation"] == 0)).sum()))
print("effectif NO IE {}".format(((data["immediate_extubation"] == 0) & (data["secondary_intubation"] == 0)).sum()))
print("effectif Auto IE {}".format(((data["immediate_extubation"] == 0) & (data["secondary_intubation"] == 1)).sum()))
print('\n')
print("Missing values in the target \n \n", data[["immediate_extubation", "secondary_intubation"]].isnull().sum())

effectif unsuccessful IE 16
effectif successful IE 119
effectif NO IE 239
effectif Auto IE 34


Missing values in the target 
 
 immediate_extubation    1
secondary_intubation    3
dtype: int64


In [171]:
data.drop(['secondary_intubation','immediate_extubation'], inplace = True, axis = 1 )

### select subset of patients

In [172]:
data = data[data.target.notnull()]

In [173]:
fct_temp = pd.read_csv('../../../data/fct_temperature.csv')
fct_temp['numero'] = fct_temp['id_patient']
fct_temp = fct_temp.drop_duplicates(['numero'])
n = pd.merge(data, fct_temp, how = 'inner', on = 'numero').shape[0]


"Nombre de match entre les tables fct_ et dim_patient {}".format(n)

'Nombre de match entre les tables fct_ et dim_patient 74'

In [174]:
print("Dataset shape :  {}" .format(data.shape))
data.target.value_counts()/len(data) * 100

Dataset shape :  (135, 43)


successful IE      88.148148
unsuccessful IE    11.851852
Name: target, dtype: float64

In [175]:
data = data.replace("NF", np.nan)

# Export

In [176]:
data.to_csv('../../../data/data_merged.csv' , index = False)